# 1. Example to grap images from a webcam
(but this only gives you an rgb, not depth)
hit 'q' in the window to stop

[ WARN:0@0.747] global cap_v4l.cpp:982 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@0.747] global obsensor_uvc_stream_channel.cpp:156 getStreamChannelGroup Camera index out of range


TypeError: object of type 'NoneType' has no len()

# 2. Color based patch segmentation

In [ ]:
import os
os._exit(0)

In [24]:
import cv2 as cv
import numpy as np
from robotic import ry
import time
print(cv.__version__)

4.7.0


In [91]:
C = ry.Config()
C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandasTable.g'))

obj = C.addFrame('obj')
obj.setPose('t(0. 0.1 0.8)')
obj.setShape(ry.ST.ssBox, size=[.05,.05,.05,.005])
obj.setColor([1,.0,0])
obj.setMass(.1)
obj.setContact(True)
C.view()

0

In [92]:
bot = ry.BotOp(C, False)
bot.home(C)

In [93]:
fxypxy = bot.getCameraFxypxy("camera")
print(fxypxy)
cameraFrame = C.getFrame("camera")

[rai] bot.cpp:getCameraFxypxy:309(0) creating camera sensor 'camera'
[322.19998169 322.19998169 320.         180.        ]


In [94]:
points = []
tau = .01

for t in range(30):
    time.sleep(0.1)

    #grab sensor readings from the simulation
    [rgb, depth] = bot.getImageAndDepth('camera')  #we don't need images with 100Hz, rendering is slow
            
    if len(rgb)>0:
        color = cv.cvtColor(rgb, cv.COLOR_BGR2RGB)
        #cv.imshow('OPENCV - rgb', color)
        
        # find redish pixel
        color_HSV = cv.cvtColor(color, cv.COLOR_BGR2HSV)
        result = color_HSV.copy()
        lower1 = np.array([0,15,10])
        upper1 = np.array([30,255,255])
        mask1 = cv.inRange(color_HSV, lower1, upper1)
        
        lower2 = np.array([330,15,10])
        upper2 = np.array([360,255,255])
        mask2 = cv.inRange(color_HSV, lower2, upper2)
        
        # display binary mask
        result[np.where(mask1==0) and np.where(mask2==0)] = 0
        result[np.where(mask1!=0) or  np.where(mask2!=0)] = 255
        cv.imshow('OPENCV - binary mask', result)
        
        # segementation
        imgray = cv.cvtColor(result, cv.COLOR_BGR2GRAY)
        contours, _ = cv.findContours(imgray, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
        
        # display segements
        cv.imshow('OPENCV - rgb contours', cv.drawContours(color, contours, -1, (0,255,0), 1))
        
        # print center of segment
        for c in contours:
            #if cv.contourArea(c) <= 50 :
            #    continue    
            x,y,w,h = cv.boundingRect(c)
            center = (x,y)
            print (center)
    
    #if len(depth)>0: cv.imshow('OPENCV - depth', 0.5* depth)

    if cv.waitKey(1) & 0xFF == ord('q'):
        break
        
    bot.sync(C)

(316, 276)
(316, 276)
(316, 276)
(316, 276)
(316, 276)
(316, 276)
(316, 276)
(316, 276)
(316, 276)
(316, 276)
(316, 276)
(316, 276)
(316, 276)
(316, 276)
(316, 276)
(316, 276)
(316, 276)
(316, 276)
(316, 276)
(316, 276)
(316, 276)
(316, 276)
(316, 276)
(316, 276)
(316, 276)
(316, 276)
(316, 276)
(316, 276)
(316, 276)
(316, 276)


In [95]:
cv.destroyAllWindows()
del bot
del C

[rai] bot.cpp:~BotOp:87(0) shutting down BotOp
[rai] simulation.cpp:~BotThreadedSim:60(0) shutting down SimThread
[rai] simulation.cpp:~Simulation:153(0) shutting down Simulation


# 3. example to use multiple camera attached to different robot frames

In [ ]:
import os
os._exit(0)

In [13]:
from robotic import ry
import numpy as np

In [14]:
C = ry.Config()
C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandasTable.g'))
# change the position of the central sensor
f = C.frame("camera")
f.setPosition(f.getPosition()+[0,0,.5])
# add a frame for the additional camera
f = C.addFrame("r_gripperCamera", "r_gripper")
f.setShape(ry.ST.camera, [.1])
f.addAttributes({'focalLength':0.895, 'width':640., 'height':360.})

In [15]:
bot = ry.BotOp(C, False)
bot.home(C)

In [16]:
img, _ = bot.getImageAndDepth('camera')

[rai] bot.cpp:getImageAndDepth:294(0) creating camera sensor 'camera'


In [17]:
img, _ = bot.getImageAndDepth('r_gripperCamera')

[rai] bot.cpp:getImageAndDepth:294(0) creating camera sensor 'r_gripperCamera'


In [18]:
del bot
del C

[rai] bot.cpp:~BotOp:87(0) shutting down BotOp
[rai] simulation.cpp:~BotThreadedSim:60(0) shutting down SimThread
[rai] simulation.cpp:~Simulation:153(0) shutting down Simulation
